## Setup Environment

In [ ]:
import pandas as pd
import ecoscope
import os
import fsspec
import json

ecoscope.init(selenium=True)

## Parameters

In [ ]:
config_path = ""

In [ ]:
with fsspec.open(config_path, mode='rt',) as file:
    config = json.loads(file.read())

config

In [ ]:
# EarthRanger Credentials
ER_SERVER = "https://mep-dev.pamdas.org"
ER_USERNAME = "yun.wu"
ER_PASSWORD = os.getenv("ER_PASSWORD", "")

# SubjectGroup
subjectgroup_name = "Elephants"

# Analysis Time Range
start_date = config.get('start_date')
end_date = config.get('end_date')

# Relocations Bounding Box Filter Coordinates
relocs_coord_filter_min_x = -5.0
relocs_coord_filter_max_x = 1.0
relocs_coord_filter_min_y = 12.0
relocs_coord_filter_max_y = 18.0

# Relocations Filter Point Coordinates
relocs_filter_point_coords = [[180, 90], [0, 0]]

# Trajectory Filter Settings
traj_min_length_meters=0.0
traj_max_length_meters=10000
traj_min_time_secs=120.0
traj_max_time_secs=4 * 60 * 60
traj_min_speed_kmhr=0.000001
traj_max_speed_kmhr=8.0

## Logic

### Connect to EarthRanger

In [ ]:
er_io = ecoscope.io.EarthRangerIO(
    server=ER_SERVER,
    username=ER_USERNAME,
    password=ER_PASSWORD,
    tcp_limit=5,
    sub_page_size=4000,
)

### Relocations

In [ ]:
%%time
relocs = er_io.get_subjectgroup_observations(
    group_name=subjectgroup_name,
    include_inactive=True,
    include_subject_details=True,
    include_source_details=False,
    include_subjectsource_details=False,
    filter=0,
    since=pd.to_datetime(start_date).isoformat(),
    until=pd.to_datetime(end_date).isoformat(),
    include_details=True,
)

In [ ]:
relocs = relocs[relocs.extra__subject__name != 'Habiba'] # temporary hack until relocs_filter method is fixed...

In [ ]:
# coord_filter = ecoscope.base.RelocsCoordinateFilter(
#     min_x=relocs_coord_filter_min_x,
#     max_x=relocs_coord_filter_max_x,
#     min_y=relocs_coord_filter_min_y,
#     max_y=relocs_coord_filter_max_x,
#     filter_point_coords=relocs_filter_point_coords,
# )
# relocs.apply_reloc_filter(coord_filter, inplace=True)
# relocs.remove_filtered(inplace=True)

In [ ]:
%%time
## Clean up columns that are not compatible with a GPKG format
relocs.drop(
    columns=relocs.columns[relocs.applymap(lambda x: isinstance(x, list)).any()],
    errors="ignore",
    inplace=True,
)

## Trajectory

In [ ]:
%%time
## create a trajectory
traj = ecoscope.base.Trajectory.from_relocations(relocs)

In [ ]:
## filter the trajectory
traj_seg_filter = ecoscope.base.TrajSegFilter(
    min_length_meters=traj_min_length_meters,
    max_length_meters=traj_max_length_meters,
    min_time_secs=traj_min_time_secs,
    max_time_secs=traj_max_time_secs,
    min_speed_kmhr=traj_min_speed_kmhr,
    max_speed_kmhr=traj_max_speed_kmhr,
)
traj.apply_traj_filter(traj_seg_filter, inplace=True)
traj.remove_filtered(inplace=True)

In [ ]:
%%time
## remove any incompatible columns for geopackage export
traj.drop(
    columns=traj.columns[traj.applymap(lambda x: isinstance(x, list)).any()],
    errors="ignore",
    inplace=True,
)

## Outputs

In [ ]:
# Calculate the number of rows in the geodatabse
len(relocs)

In [ ]:
len(traj)

In [ ]:
# Map of the data
m = ecoscope.mapping.EcoMap(static=True)
m.add_tile_layer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    name="Google Satellite",
    attribution="Google",
    opacity=1.0,
)
m.add_speedmap(trajectory=traj, classification_method="equal_interval", num_classes=6, bins=None)
m.add_north_arrow(position="topright")
m.add_title(title="Elephant Speed Map", align="center", font_size="18px")
m.zoom_to_gdf(traj)
m

In [ ]:
# Export to HTML
source_file_name = "/tmp/ecomap.html"
m.to_html(source_file_name)

In [ ]:
# Upload to Google Cloud Storage
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.get_bucket("us-central1-ecoscope-pipeli-92c4ecc9-bucket")
blob = bucket.blob("website/ecomap.html")

blob.upload_from_filename(source_file_name)

print('File {} uploaded to {}.'.format(
    source_file_name,
    "website"))